In [1]:
# -*- coding: utf-8 -*-

'''
##############################################################################
This script has the purpose of parsing a page from yellowpages.com and saving
the content to a JSON file.
'''
import requests
from bs4 import BeautifulSoup

# Initialize lists
dealerList, nameList, addressList, localityList = [], [], [], []
regionList, postalCodeList, telephoneList, urlList = [], [], [], []
categoriesList = []

# Specify the url
YP_base = "http://www.yellowpages.com"
search = "/search?search_terms=piano&geo_location_terms="
location = "New+York%2C+NY"

# http://www.yellowpages.com/search?search_terms=piano&geo_location_terms=New+York%2C+NY

locationList = ["New+York%2C+NY"]

testLink = "http://www.yellowpages.com/search?search_terms=piano&geo_location_terms=New%20York%2C%20NY&page=7"
#"Seattle%2C+WA", "Los+Angeles%2C+CA"

# Query the website and return the html to the variable 'page'
# page = requests.get(YP_base + search + location)

# Parse the html in the 'page' variable, and store it in Beautiful Soup format
# soup = BeautifulSoup(page.text, "lxml")

In [31]:
def scrapSite():
    scrap = []    
    for loc in locationList:
        
        #inside location
        page = requests.get(YP_base + search + loc)   
        soup = BeautifulSoup(page.text, "lxml")
        scrap += scrapPage(soup)
        
        #move to next page in pagination
        while True:
            try:
                tag = soup.find(class_="next ajax-page")
                nextSearch = tag['href']
            except TypeError:
                break
            
            page = requests.get(YP_base + nextSearch)
            soup = BeautifulSoup(page.text, "lxml")
            scrap += scrapPage(soup)
            
    return scrap
            

In [30]:
def scrapPage(soup):
    
    infoTag = soup.find_all('div')
    returnList = []
    
    for el in infoTag: 
        
        tempList = []
        
        try:
            if el['class'] == ['info']:
                
                try:
                    name = el.find(itemprop="name")
                    tempList.append(name.text)
                except AttributeError:
                    tempList.append(name.text)

                try:
                    address = el.find(itemprop="streetAddress")
                    tempList.append(address.text)
                except AttributeError:
                    tempList.append('NA')

                try:
                    locality = el.find(itemprop="addressLocality")
                    tempList.append(locality.text)
                except AttributeError:
                    tempList.append('NA')

                try:
                    region = el.find(itemprop="addressRegion")
                    tempList.append(region.text)
                except AttributeError:
                    tempList.append('NA')

                try:
                    postalCode = el.find(itemprop="postalCode")
                    tempList.append(postalCode.text)
                except AttributeError:
                    tempList.append('NA')

                try:
                    telephone = el.find(itemprop="telephone")
                    tempList.append(telephone.text)
                except AttributeError:
                    tempList.append('NA')

                url = el.find(class_="track-visit-website")
                try:
                    tempList.append(url['href'])
                except TypeError:
                    tempList.append('NA')

                try:
                    categories = el.find(class_="categories")
                    tempList.append(categories.text)
                except:
                    tempList.append('NA')
            
            if tempList:
                returnList.append(tempList)

        except KeyError:
            pass
    
    return returnList
   

In [33]:
import csv


filename = 'firstTest.csv'

scrap = scrapSite()

def save_file(scrap, filename):

	with open(filename, 'wb') as csvfile:
	        
	        writer = csv.writer(csvfile, delimiter=',',)
	    
	        writer.writerow(['Name', 'Address', 'Locality',
	        				 'Region', 'Postal Code', 'Telephone',
	        				 'Website', 'Category'
	        ])
	        for list in scrap:
	            writer.writerow(list)

In [19]:
# def scrapPage(soup):
#     infoTag = soup.find_all('div')
#     for el in infoTag: 
#         try:
#             if el['class'] == ['info']:
                
#                 name = el.find(itemprop="name")
#                 nameList.append(name.text)

#                 try:
#                     address = el.find(itemprop="streetAddress")
#                     addressList.append(address.text)
#                 except AttributeError:
#                     addressList.append('NA')

#                 try:
#                     locality = el.find(itemprop="addressLocality")
#                     localityList.append(locality.text)
#                 except AttributeError:
#                     localityList.append('NA')

#                 try:
#                     region = el.find(itemprop="addressRegion")
#                     regionList.append(region.text)
#                 except AttributeError:
#                     regionList.append('NA')

#                 try:
#                     postalCode = el.find(itemprop="postalCode")
#                     postalCodeList.append(postalCode.text)
#                 except AttributeError:
#                     postalCodeList.append('NA')

#                 try:
#                     telephone = el.find(itemprop="telephone")
#                     telephoneList.append(telephone.text)
#                 except AttributeError:
#                     telephoneList.append('NA')

#                 url = el.find(class_="track-visit-website")
#                 try:
#                     urlList.append(url['href'])
#                 except TypeError:
#                     urlList.append('NA')

#                 try:
#                     categories = el.find(class_="categories")
#                     categoriesList.append(categories.text)
#                 except:
#                     categoriesList.append('NA')

#         except KeyError:
#             pass
   


In [26]:
scrap = scrapSite()

In [29]:
print len(scrap)
for el in scrap:
    print el
    print

185
[u'Piano', u'214A W 39th St', u'New York,\xa0', u'NY', u'10018', u'(212) 764-8444', 'http://www.pianosnyc.com', u'Pianos & Organs']

[u'Allegro Pianos', u'205 W 58th St', u'New York,\xa0', u'NY', u'10019', u'(212) 265-2455', 'http://www.allegropianos.com', u'Pianos & OrgansMusical Instruments']

[u'Richmond Pianos', u'261 W 35th St', u'New York,\xa0', u'NY', u'10001', u'(212) 736-0867', 'NA', u'Pianos & OrgansMusical Instruments']

[u'Frank & Camilles Keyboard Center of West 57th St', u'29 W 57th St', u'New York,\xa0', u'NY', u'10019', u'(212) 832-8289', 'http://www.frankandcamilleswest.com', u'Pianos & OrgansPiano & Organ Moving']

[u'Piano Games', u'244 W72nd St, #12E', u'New York,\xa0', u'NY', u'10023', u'(917) 749-2117', 'http://riversidepiano.com', u'Pianos & Organs']

[u'Beethoven Pianos Inc', u'211 W 58th St Apt 1', u'New York,\xa0', u'NY', u'10019', u'(212) 765-7300', 'http://www.beethovenpianos.com', u'Pianos & OrgansPiano & Organ Moving']

[u'The Piano Studio', u'291 Broa